#### Changing the directory

In [2]:
%pwd

'd:\\Office\\ODCRU\\02_During_Prep_Prog\\ODCRU-AI-Projects\\experiment'

In [3]:
import os
os.chdir("../")
%pwd

'd:\\Office\\ODCRU\\02_During_Prep_Prog\\ODCRU-AI-Projects'

#### Use of ConfigBox

In [4]:
dicta = {"a":1,"b":2}
dicta["a"]

1

In [ ]:
# dicta.a   ### AttributeError as dict does not support attribute-style access 
            ### so we need to use a custom class or a library like box

AttributeError: 'dict' object has no attribute 'a'

In [5]:
from box import ConfigBox

dicta = ConfigBox(dicta)
dicta.a

1

#### Reading the YAML File

In [6]:
import re
def resolve_config(content: dict):
    '''
    This function resolves the placeholders in the configuration dictionary.
    Placeholders are in the format ${key1.key2} and will be replaced with the
    corresponding value from the config dictionary.
    Args:
        config (dict): The configuration dictionary with potential placeholders.
    Errors:
        Exception: Raises an exception if a placeholder cannot be resolved.
    Returns:
        dict: The configuration dictionary with placeholders resolved.
        
    '''
    pattern = re.compile(r"\$\{([^}^{]+)\}")

    def replacer(match):
        try:
            keys = match.group(1).split(".")
            value = content
            for k in keys:
                value = value[k]
            return str(value)
        except Exception as e:
            raise e

    def walk(d):
        for k, v in d.items():
            if isinstance(v, dict):
                walk(v)
            elif isinstance(v, str):
                d[k] = pattern.sub(replacer, v)

    walk(content)
    return content

In [7]:
import yaml

def read_yaml(path_to_yaml: str) -> ConfigBox:
    """Reads a yaml file and returns a ConfigBox object.
    Args:
        path_to_yaml (str): path like input
    Errors:
        exception: Empty file or any other exception while reading yaml file
    Returns:
        ConfigBox: A ConfigBox object containing the loaded YAML data
    """
    try:
        with open(path_to_yaml) as yaml_file:
            content = yaml.safe_load(yaml_file)
            content = resolve_config(content)
            return ConfigBox(content)
    except Exception as e:
        raise e  
    
    

In [8]:
path_to_yaml = "config/config.yaml"
config = read_yaml(path_to_yaml)
config

ConfigBox({'artifacts_root': 'artifacts', 'yolo8': {'model_name': 'yolov8n.pt', 'model_type': 'pretrained_models', 'model_parent_folder': 'artifacts/pretrained_models', 'model_saved_path': 'artifacts/pretrained_models/yolov8n.pt'}, 'trained_yolo8_model': {'model_name': 'best.pt', 'model_type': 'trained_yolo8_model', 'model_parent_folder': 'artifacts/trained_yolo8_model/weights', 'model_saved_path': 'artifacts/trained_yolo8_model/weights/best.pt'}, 'inference_yolo8_model': {'model_path': 'artifacts/trained_yolo8_model/weights/best.pt'}})

In [9]:

config.yolo8.model_saved_path

'artifacts/pretrained_models/yolov8n.pt'

#### Load the Model

In [12]:
from ultralytics import YOLO

def load_pretrained_model(model_path: str):
    model = YOLO(model_path)
    return model

In [11]:
pretrained_model = load_pretrained_model(config.YOLO8.model_saved_path)
pretrained_model

YOLO(
  (model): DetectionModel(
    (model): Sequential(
      (0): Conv(
        (conv): Conv2d(3, 16, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(16, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (1): Conv(
        (conv): Conv2d(16, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
        (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
        (act): SiLU(inplace=True)
      )
      (2): C2f(
        (cv1): Conv(
          (conv): Conv2d(32, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_stats=True)
          (act): SiLU(inplace=True)
        )
        (cv2): Conv(
          (conv): Conv2d(48, 32, kernel_size=(1, 1), stride=(1, 1), bias=False)
          (bn): BatchNorm2d(32, eps=0.001, momentum=0.03, affine=True, track_running_s

#### Train the Model

In [9]:
path_to_yaml = "config/params.yaml"
params = read_yaml(path_to_yaml)
params

ConfigBox({'artifacts_root': 'artifacts', 'image_data': {'data_parent_folder': 'data', 'data_file_name': 'data.yaml', 'data_file_path': 'data/data.yaml'}, 'training_params_yolo8': {'data': 'data/data.yaml', 'epochs': 10, 'img_size': 416, 'batch_size': 2, 'device': 0, 'project': 'artifacts', 'name': 'trained_yolo8_model', 'exist_ok': True, 'workers': 0, 'mosaic': 0, 'amp': False}, 'evaluate_params_yolo8': {'data': 'data/data.yaml', 'img_size': '416', 'batch_size': '2', 'device': '0', 'project': 'artifacts', 'name': 'trained_yolo8_model_val', 'save': True, 'plots': True, 'metrics_filename': 'metrics.json', 'final_plot_name': 'final_metrics.png', 'metrics_file_path': 'artifacts/trained_yolo8_model_val/metrics.json', 'final_plot_path': 'artifacts/trained_yolo8_model_val/final_metrics.png'}, 'inference_params_yolo8': {'img_size': '416', 'confidence_threshold': 0.4, 'iou_threshold': 0.5, 'device': '0', 'save': True, 'save_txt': True, 'save_conf': True, 'project': 'artifacts', 'name': 'infere

In [36]:
def train_model(pretrained_model: object, params: ConfigBox):
    device = params.device if params.device != "cuda:0" else "0"
    train_model = pretrained_model.train(
        data=params.data,
        epochs=params.epochs,
        imgsz=params.img_size,
        batch=params.batch_size,
        device=device,
        project=params.project, 
        name=params.name,
        workers=params.workers,
        exist_ok=params.exist_ok
    )
    

    return train_model
    
    

In [17]:
trained_model = train_model(pretrained_model = pretrained_model, params= params.training_params_yolo8)
trained_model

Ultralytics 8.3.252  Python-3.13.2 torch-2.9.1+cu126 CUDA:0 (NVIDIA GeForce GTX 1650, 4096MiB)
engine\trainer: agnostic_nms=False, amp=True, augment=False, auto_augment=randaugment, batch=4, bgr=0.0, box=7.5, cache=False, cfg=None, classes=None, close_mosaic=10, cls=0.5, compile=False, conf=None, copy_paste=0.0, copy_paste_mode=flip, cos_lr=False, cutmix=0.0, data=data/data.yaml, degrees=0.0, deterministic=True, device=0, dfl=1.5, dnn=False, dropout=0.0, dynamic=False, embed=None, epochs=10, erasing=0.4, exist_ok=True, fliplr=0.5, flipud=0.0, format=torchscript, fraction=1.0, freeze=None, half=False, hsv_h=0.015, hsv_s=0.7, hsv_v=0.4, imgsz=512, int8=False, iou=0.7, keras=False, kobj=1.0, line_width=None, lr0=0.01, lrf=0.01, mask_ratio=4, max_det=300, mixup=0.0, mode=train, model=artifacts/pretrained_models/yolov8n.pt, momentum=0.937, mosaic=1.0, multi_scale=False, name=trained_yolo8_model, nbs=64, nms=False, opset=None, optimize=False, optimizer=auto, overlap_mask=True, patience=100, 

ultralytics.utils.metrics.DetMetrics object with attributes:

ap_class_index: array([0])
box: ultralytics.utils.metrics.Metric object
confusion_matrix: <ultralytics.utils.metrics.ConfusionMatrix object at 0x00000242F9863110>
curves: ['Precision-Recall(B)', 'F1-Confidence(B)', 'Precision-Confidence(B)', 'Recall-Confidence(B)']
curves_results: [[array([          0,    0.001001,    0.002002,    0.003003,    0.004004,    0.005005,    0.006006,    0.007007,    0.008008,    0.009009,     0.01001,    0.011011,    0.012012,    0.013013,    0.014014,    0.015015,    0.016016,    0.017017,    0.018018,    0.019019,     0.02002,    0.021021,    0.022022,    0.023023,
          0.024024,    0.025025,    0.026026,    0.027027,    0.028028,    0.029029,     0.03003,    0.031031,    0.032032,    0.033033,    0.034034,    0.035035,    0.036036,    0.037037,    0.038038,    0.039039,     0.04004,    0.041041,    0.042042,    0.043043,    0.044044,    0.045045,    0.046046,    0.047047,
          0.0480

#### Evaluate Model

In [ ]:
def evaluate_model(trained_model: object, params: ConfigBox):
    results = trained_model.val(
        data=params.data,
        img_size=params.img_size,
        batch_size=params.batch_size,
        device=params.device,
        project=params.project,
        name=params.name,
        save=params.save,
        plots=params.plots
    )
    return results

True

#### Inferencing

In [ ]:
import cv2

def run_inferencing(trained_model: object, params: ConfigBox, image_path: str):
    model = YOLO(trained_model)
    results = model(
        source=image_path,
        # img_size=params.img_size,
        conf=float(params.confidence_threshold),
        show=True,
        iou=params.iou_threshold,
        device=params.device,
        save=params.save,
        save_txt=params.save_txt,
        save_conf=params.save_conf,
        project=params.project,
        name=params.name,
        exist_ok=params.exist_ok
    )


In [17]:

img_path = "data/test/images/Cars71.png"
run_inferencing(
    trained_model=config.inference_yolo8_model.model_path,
    params=params.inference_params_yolo8,
    image_path=img_path
)


image 1/1 d:\Office\ODCRU\02_During_Prep_Prog\ODCRU-AI-Projects\data\test\images\Cars71.png: 256x416 1 license_plate, 60.3ms
Speed: 1.4ms preprocess, 60.3ms inference, 27.3ms postprocess per image at shape (1, 3, 256, 416)
